In [7]:
#входные данные 
#директория со всеми данными
directory = 'C:/Users/Admin/OpenCV/pictures'

#Название аудиофайла
audioFile = 'music.mp3'

#Название видеофайла
videoFile = 'Video.mp4'

#название конечного файла
result = 'Final_video_audio.mp4'

In [8]:
#импорт необходимых библиотек
#для работы с opencv
import cv2
#для работы с опер.сист.
import os
from os.path import join
#для работы с массивами 
import numpy as np

In [9]:
#функция создания слайдшоу + переходы
#на входе (путь до картинки, путь конечного сохранения, кол-во фреймов/сек, время на одну картинку)
#на выходе mp4 файл
def convert_pictures_to_video(pathIn, pathOut, fps, time):
    #массив с исходными картинками
    pictures_array = []
    #массив с переделанными фреймами из картинок
    frame_array = []
    #считывание картинок из файла (с расширением .jpg)
    jpg_files = [f for f in os.listdir(pathIn) if os.path.splitext(f)[1] == '.jpg']
    #проход по каждой картинке из файла
    for i in range(len(jpg_files)):
        #путь до картинки
        filename = pathIn + jpg_files[i]
        #считывание картинки
        img = cv2.imread(filename)
        #изменение размера каритнки до 800х800
        img = cv2.resize(img, (800, 800))
        #сохранение высоты, ширины, количества слоев(RGB=3)
        height, width, layers = img.shape
        #сохранение размера картинки в отдельную переменную
        size = (width, height)
        
        #добавление картинки в массив pictures_array 
        pictures_array.append(img)
        
        #добавление необходимово кол-ва фреймов в конечный массив для первой картинки, иначе для нее будет мало времени
        if i==0:
            for k in range(time):
                frame_array.append(img)
        #добавление переходов
        else:
            
            #переход вправо
            if i%5 == 1:
                for m in range(0,800, 10):
                    #конечная ширина обрезки новой картинки смещается  вправо
                    width1, height1 = 800-m, 800
                    x1, y1 = 0, 0
                    width2, height2 = 800, 800
                    #начальное положение обрезки старой картинки смещается вправо
                    x2, y2 = 800-m, 0
                    
                    #обрезка картинок
                    crop_img1 = pictures_array[i][y2:y2+height2, x2:x2+width2]
                    crop_img2 = pictures_array[i-1][y1:y1+height1, x1:x1+width1]
                    
                    #соединение двух картинок по горизонтали
                    res = np.concatenate((crop_img1, crop_img2), axis=1)
                    #добавление этой конкатенации в конечный массив
                    frame_array.append(res)
                    
            #плавный переход
            elif i%5 == 2:
                for m in range(100):
                    #коэффициенты прозрачности наложения одной картинки на другую
                    alpha = m/100
                    beta = 1.0 - alpha
                    #наложение одной каритнки на другую с некоторой прозрачностью
                    res = cv2.addWeighted(pictures_array[i], alpha, pictures_array[i-1], beta, 0.0)
                    frame_array.append(res)
            
            #переход влево
            elif i%5 == 3:
                for m in range(0,800, 10):
                    width1, height1 = 800-m, 800
                    x1, y1 = 0, 0
                    width2, height2 = 800, 800
                    x2, y2 = 800-m, 0
                    crop_img1 = pictures_array[i][y2:y2+height2, x2:x2+width2]
                    crop_img2 = pictures_array[i-1][y1:y1+height1, x1:x1+width1]
                    
                    #соединение двух картинок как в переходе вправо, но наоборот
                    res = np.concatenate((crop_img2, crop_img1), axis=1)
                    
                    frame_array.append(res)
                    
            #переход вниз
            elif i%5 == 4:
                for m in range(0,800, 10):
                    #конечная высота обрезки новой картинки смещается вниз
                    width1, height1 = 800, 800-m
                    x1, y1 = 0, 0
                    width2, height2 = 800, 800
                    #начальное положение обрезки старой картинки смещается вниз
                    x2, y2 = 0, 800-m
                    crop_img1 = pictures_array[i][y2:y2+height2, x2:x2+width2]
                    crop_img2 = pictures_array[i-1][y1:y1+height1, x1:x1+width1]
                    
                    #соединение картинок по вертикали
                    res = np.concatenate((crop_img1, crop_img2), axis=0)

                    frame_array.append(res)
            
            #переход вверх
            elif i%5 == 0:
                for m in range(0,800, 10):
                    width1, height1 = 800, 800-m
                    x1, y1 = 0, 0
                    width2, height2 = 800, 800
                    x2, y2 = 0, 800-m
                    crop_img1 = pictures_array[i][y2:y2+height2, x2:x2+width2]
                    crop_img2 = pictures_array[i-1][y1:y1+height1, x1:x1+width1]
                    
                    #соединение двух картинок как в переходе вниз, но наоборот
                    res = np.concatenate((crop_img2, crop_img1), axis=0)

                    frame_array.append(res)
            
            #добавление фреймов оригинальной картинки в конечный массив в зависимости от времени
            for k in range(time):
                frame_array.append(img)
                
    #инициализироуем видео-мэйкер с параметрами выходного пути, кодека, фпс, размера файла
    output = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    
    #преобразовываем каждый фрейм в видео 
    for i in range(len(frame_array)):
        output.write(frame_array[i])
    
    #релиз видео
    output.release()

In [10]:
pathIn = directory + '/'

#название выходного файла-слайдшоу (без видео и аудио)
outputFile_1 = 'Slideshow.mp4'

#конечный путь для выходного файла-слайдшоу
pathOut = pathIn + outputFile_1

#количество фреймов в сек.
fps = 50

#время просмотра одной картинки -- здесь 2 сек.
time = fps*2

In [11]:
#вызов функции
convert_pictures_to_video(pathIn, pathOut, fps, time)

In [12]:
#импорт библиотек и модулей для работы с конкатенацией видео и наложения аудио
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, CompositeAudioClip
#модуль для работы с громкостью звука
from moviepy.audio.fx.volumex import volumex

In [13]:
#создание клипа из аудиофайла
audioClip = AudioFileClip(pathIn + audioFile)
#уменьшение громкости звука на 50%
audioClip = audioClip.fx(volumex, 0.5)
#создание клипа из файла-слайдшоу
slideshowVideo = VideoFileClip(pathIn + outputFile_1)
#создание клипа из данного видеофайла
originalVideo = VideoFileClip(pathIn + videoFile).subclip(0,audioClip.duration - slideshowVideo.duration)

#конкатенация файла-слайдшоу и данного видеофайла
finalVideo = concatenate_videoclips([slideshowVideo, originalVideo], method="compose")

#название сконкатенированного файла
outputFile_2 = 'Final_video.mp4'

#запись финального видео без аудио в папку
finalVideo.write_videofile(pathIn + outputFile_2)

Moviepy - Building video C:/Users/Admin/OpenCV/pictures/Final_video.mp4.
MoviePy - Writing audio in Final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C:/Users/Admin/OpenCV/pictures/Final_video.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/Admin/OpenCV/pictures/Final_video.mp4


In [14]:
#композиция аудио из видео и аудиоклипа
finalVideoAudio = CompositeAudioClip([finalVideo.audio, audioClip])
#наложение аудио на финальное видео
finalVideoAudio = finalVideo.set_audio(finalVideoAudio)
#запись конечного файла в папку
finalVideoAudio.write_videofile(pathIn + result)

Moviepy - Building video C:/Users/Admin/OpenCV/pictures/Final_video_audio.mp4.
MoviePy - Writing audio in Final_video_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C:/Users/Admin/OpenCV/pictures/Final_video_audio.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/Admin/OpenCV/pictures/Final_video_audio.mp4
